# Case study: Herd Behaviour in Markets

*Ammar Abu Shamleh*

### 1. Introduction
Herd behaviour refers to an intrinsic tendency humans have to mimic the movements of a larger group, whether or not those movements are rational. Those who engage in herd behaviour often exhibit behaviour they would not normally exhibit themselves, were they to act purely on their own decision-making criteria. 

Stock and currency markets exhibit especially interesting forms and consequences of herd behaviour. Herd mentality can drive investors to make decisions about entering or leaving markets that they wouldn't normally make with their own individual judgment. 

Another interesting phenomenon is that of 'follow the leader behaviour', where individuals within a market will make judgments based on the actions/stances taken by well known, "big name" investors or speculators.

This kind of socially reactive behaviour can drive markets in dangerous and unpredictable directions, and has been the underlying cause of many infamous and disastrous market crashes, in both recent and distant history. Many of these crashes are termed "speculative bubbles" or "stock market bubbles", because they involve something (such as a share price or currency value) being driven beyond its intrinsic value - by the aforementioned behaviour - before collapsing; similar to how a bubble expands and bursts. 

Interestingly, the socially reactive behaviour described here drives both the initial expansion and the collapse of the bubble, as those fleeing the market after the early signs of value depreciation trigger reactions in others, who then leave the market; this creates a knock-on or "domino" effect, which drives the collapse. One prominent example (among many) is the *Dot-com bubble*, which collapsed during 2000-2002.

#### 1.1 Follow the Leader
A related, but slightly different behaviour, is that of following prominent figures, and mimicking their movements and decisions. Similar to herd behaviour, this often results people making decisions and exhibiting behaviour uncharacteristic of their normal behaviours. 

This can also interact with herd behaviour, in the case where people follow the movements of a large group, and are simultaneously influenced by prominent figures internal or external to the group. In such a sense, people would observe the actions of those around them, while also observing (and placing more emphasis on) the actions of larger prominent figures.

### 2. Model Overview
I created a simplistic model of the stock market with the aim of observing the effects of the two types of behaviours described above. The goal here was not to create a model useful for discussion of stock markets, but rather to create a model useful for discussion of these specific behaviours; the stock market is simply a valuable means for exploring them.

#### 2.1 Investors as Agents
Applying the behaviours described above to a simulation of investors engaging with the stock market lends itself well to an **agent-based model**, where each investor is an agent exhibiting relatively simplistic behaviour, with the collective behaviour of all agents driving the market. Such is how I chose to build my model.

The premise of the model is that a simulation runs over a number of time-steps, and from each time-step to the next, each investor will decide whether to leave or join the market, by considering a number of factors. The factors considered, and the weight given to each factor, define the investor's behaviour. All investors within the model are agents, and behave according to the same rules. There is some necessary randomness built into the behaviour of investors, in order to capture the undpredictability of real human behaviour, and differentiate different investors from each other in subtle ways. Investors are modeled as *Investor* objects.

Investors leave or join the market by buying or releasing shares. Before the simulation begins, each investor is assigned a *number of shares*, defining the quanity in which they purchase/release shares. For instance, an investor assigned the number 5 will purchase 5 shares when entering the market, and release exactly 5 shares when leaving. They will not trade in any other quantity, and the quantity does not change over the course of the simulation. Investors are thus either completely *in* or *out* of the market. This number is probabilistically affected by the number of conncetions the investor has in the social sphere (*Section 2.3*).

The factors influencing the chance for an investor to join/leave the market are:
- Other investors connected to this investor in the social sphere: whether or not they're in the market; whether they've left/joined the market recently; how many connections they have
- How the market has changed recently
- Whether the market is approaching it's limit (*Section 2.2*)
- Whether the investor in question has previously left the market during the simulation

The principal methods of interest supplied by an *Investor* object are:
- *probToLeave*: Returns a probability for the investor to leave the market between the current time-step and the next 
- *probToJoin*: Returns a probability for the investor to join the market between the current time-step and the next

An investor is either in or out of the market, and so during each time-step, the appropriate method would be called, depending on which of the two states they're in. At the start of the market, investors are randomly added to the market with a starting probability *p*, which is a parameter of the model.

#### 2.2 The Market
The market itself is modeled extremely simplistically. It is simply an entity with a number of shares currently in posession of investors, and a limit. The limit defines how many shares may be in the posession of investors at any one time. This is reflective of the fact that any share/asset only exists in a limited quantity, and the relation of the demand for the asset to the quantity suppliable determines its price. 

The number of shares in posession of investors change as investors enter/exit the market, with the number of shares:
- Increasing in the former case (as an investor has purchased and now owns a number of shares), and
- Decreasing in the latter case (as an investor has cashed in their posessed shares, which re-enter the market)

The market is modeled as an object, called a *Market*.

It is important to note that this model does not accurately reflect the precise financial behaviour and mathematics that underpins stock markets. However, these simplifications are made because the diveregence from real financial mathematics has no significance to the behaviour of interest. The mathematical effect of an investor joining/leaving the market, as it's modeled here, and when compared with real markets, still has the same ultimate effect of an increase/decrease in share price, and drives the behaviours of other investors in the same way. Thus the abstraction away from reality is a justifiable (and necessary) one.

#### 2.3 Investors in a Social Sphere
Investors are connected to one other in a social sphere, which is a modeled as a graph. Each node represents an investor, and each edge represents a social connection to another investor. Investors will be influenced to join/leave the market by the actions of those they're connected to in the sphere, and the strength of this influence will be determined by the nodal degree of the investor to which they're connected. An investor with more connections will influence the actions of their connections more than an investor with fewer connections. This network is modeled as an object, called a *SocialSphere*.

#### 2.4 Simulating over Time Steps
Finally, what ties everything together is a "Runner" class, which sets up the various pieces of the model, initializes the market, and runs the simulation over a number of time steps. Below are abbreviated versions of the main loop that runs the simulation, and the *tick* method that moves the entire model one time step forward, taking various parameters that are used by the method, or used by investors to decide on a probability to leave or join the market.

In [ ]:
marketValues = []
marketValues.append(market.totalShares)
    for i in range(numTimeSteps):
        tick(market, investors, socialSphere, marketValues, i, largestNodeDegree, averageNodeDegree, herdBehaviour)
        marketValues.append(market.totalShares)
        
largestNumShares = max(marketValues)
    plt.plot(marketValues)
    plt.ylabel('Shares Purchased')
    plt.xlabel('Time Steps')
    plt.axis([0, numTimeSteps, 0, largestNumShares])
    plt.show()

In [ ]:
def tick(m, investors, s, mValues, i, largestDeg, averageDeg, herd):
    
    marketSize = len(s.g)
    numJoined = 0
    numLeft = 0
    
    for key in investors:
        investor = investors[key]
        
        if(investor.isInMarket()):
            probToLeave = investor.probToLeave(s, investors, mValues, i, m, largestDeg, averageDeg, herd)
            if(random.random() <= probToLeave and 
               (not investor.changedStanceRecently()) and 
                numLeft <= marketSize/50):
                
                numLeft = numLeft + 1
                market.removeInvestor(investor)
                investor.leaveMarket()
                
            else:
                investor.stayInMarket()
                
        else:
            probToJoin = investor.probToJoin(s, investors, mValues, i, m, largestDeg, averageDeg, herd)
            if(random.random() <= probToJoin and 
               market.canJoin(investor) and 
               (not investor.changedStanceRecently()) and 
                numJoined <= marketSize/50):
                
                market.addInvestor(investor)
                investor.enterMarket()
                numJoined = numJoined + 1
                
            else:
                investor.stayOutsideMarket()

Notably, investors who have joined/left the market will not make another action (i.e. join or leave again) until a small number of timesteps have passed, in order to prevent wild behaviour, and as a touch of realism. Also, the model only permits a fraction of all investors to join/leave within a single timestep.

### 3. Building the Social Sphere
An important decision to make in building the model is that of setting up the social sphere (the network representing all investors and their social connections with each other). As the model is studying how people are influenced by those around them, it is important that the network resemble a real social network of human connections. The two main alternatives considered by this model are the *Barabasi-Albert* and *Watts-Strogatz* models.

#### 3.1 Barabasi-Albert
The defining feature of this model is that the distribution of nodal degrees obeys a power law, and the emergence of nodes that act as hubs with very high degrees. Due to the presence of hubs with very large degrees, this model is useful for modelling "follow the leader" behaviour. In real-world terms, the hubs would represent well known traders with large numbers of followers that pay attention to their actions within the stock and currency markets. George Soros is a particularly prominent example. Due to their high nodal degrees, the actions of these nodes would have a large influence on the actions of their numerous connections, thus creating the desired "follow the leader" behaviour.

#### 3.2 Watts-Strogatz
*Watts-Strogatz* is another proposed model for creating graphs that resemble real social networks, and is thus another alternative considered. The Watts-Strogatz model, instead of producing hubs, produces tightly connected clusters, and weak connections between them. Because of the nature of this model, with high clustering and low path lengths, it is adequate for modelling herd behaviour. Each node is affected by the nodes around it, which will likely encompass most of the cluster that it belongs to. Those nodes are affected by their own connections, some of which are likely to belong to a different cluster. From the fact that path lengths are low and clustering is high, it is easy to see how herd behaviour will arise within the network.

It is interesting to note that the same principle in theory applies to the *Barabasi-Albert* model as well. However, the key difference is in the fact that investors will consider the degree of their connections in determining the weight of the connection's influence. In B-A, this will result in the hubs dominating the behaviour of the network, due to their (comparatively) extremely high degrees. Conversely, in W-S, this will result in nodes being influenced fairly equally by most/all of their connections. This is why B-A leads to follow-the-leader behaviour, while W-S leads to herd mentality.

### 4. Results
The simulation was run over 300 timesteps with various combinations of parameters, and the results displayed graphically using *pyplot*. Below are 3 such sets of results.

#### 4.1 Results from Barabasi-Albert
![Stock Market simulation with B-A model, 3000 investors](BA Model 3000.jpg)

*Figure 1* shows the effect of a Barabasi-Albert model, which is the creation, expansion and collapse of large, radical market bubbles. This is characteristic of most results generated by the simulation using this model of network construction.

Insterestingly, by analyzing when the hubs leave/join the market, a clear correlation can be observed with the transitions between peaks and troughs. At time step 24, right before the first collapse, an investor with degree 157 leaves. At timesteps 67-68, right before the second bubble begins, two investors with degrees 63 and 157 join. At timesteps 82-84, right before the second collapse, a small handful of investors with degrees between 50 and 60 leave. 

These results show that hub-dominated behaviour in markets can be radical, and often irrational, leading to unpredictable and extreme swings, and is characterized by very high peaks, very low troughs, and multiple bubbles.

#### 4.2 Results from Disabling Herd Behaviour
Following the results obtained previously, I ran the simulation with a slight change in agent behaviour: I removed all influence from social connections, and simply had investors respond to the market changes alone. The behaviour displayed was remarkably different.
![Stock Market simulation without social connections, 3000 investors](No Herd Model 3000.jpg)

While the results in *figure 2* still depict an initial expansion and collapse (as the model is initialized to be in the middle of an expanding bubble), the behaviour following the first collapse is much more stable and consistent than that seen with a *Barabasi-Albert* network. While the values fluctuate, they do so within a small window, and no bubbles follow the first. Because investors are less likely to join after initially leaving, it makes sense that a second bubble would not form, as they are no longer under the influence of hubs (which outweighs their apprehension to rejoin the market in the BA model).

#### 4.3 Results from Watts-Strogatz
![Stock Market simulation with W-S model, 3000 investors](WS Model 3000.jpg)
*Figure 3* interestingly shows that a *Watts-Strogatz* model produces results very comparable to those produced when the social network is abandoned entirely (in *Figure 2*). The erratic behaviour of the *Barabasi-Albert* model is gone. Similar to *figure 2*, following the first bubble, the values simply fluctuate within a small window, and no further bubbles form.

#### 4.4 Comparing Peak-Trough Values
One interesing observation made in figures 1, 2 and 3 is that the values seem to reach a much lower global minimum when using *Barabasi-Albert* as the underlying social network. To test this, I derived a method for computing the extremity with which the market swings from peaks to troughs.

*swing = GlobalMax - GlobalMin / MarketLimit*

This value, which I call the *swing value* , takes the difference between the global max and global min values across the simulation, and divides that difference by the market's limit (the total number of shares available within the market). Over a number of simulations for each of the 3 models above, the average swing value was taken.
![Swing values for different models](Swings.jpg)


As expected, *figure 4* shows that models underpinned by a *Barabasi-Albert* network have much higher swing values. This is because hubs can push system behaviour into extremities that would otherwise not be reached.

### 5. Conclusions
The results produced by the simulation show that hub-dominated behaviour, or "follow the leader" behaviour, can produce erratic results, and create wild and extreme transitions between peaks and troughs. BA models are characterized by the presence of multiple bubbles, and seem to have less stability. In reality, the danger with hub-dominated behaviour is the lack of safety mechanisms, as a result of the dominating influence posessed by a small number of agents within the model. When an agent that is a hub decides to leave or join the market, that action will create a dominating influence among the agent's connections, inspiring the behaviour among a large number of investors. This then propogates throughout the rest of the network, and creates a trend of movement, even when the majority of agents within the network would not normally take such behaviours. It's for this reason that more bubbles can form after the first; even though agents are inherently unlikely to rejoin after leaving (and thus after the first bubble collapse), this apprehension will be outweighed by the influence of the hub nodes having rejoined the market. Thus bubbles can very easily form and burst, and do so in extreme fashions, creating the radical behaviour seen in *figure 1*.

Conversely, the WS model requires that a large set of agents behave in a certain way in order to create a trend, due to the similar levels of influence most investors have on their social connections. Since agents are intrinsically unlikely to rejoin after the first bubble, it is near impossible for a trend to be created, and thus no more bubbles form. Thus the difference lies in the number of investors required to create a trend:
- With a *Barabasi-Albert* model, even just a handful of hub nodes can create an overwhelming trend
- With a *Watts-Strogatz* model, a sufficient enough number of investors are required in order to solidify a trend

Much of the behaviour witnessed in these systems is emergent. I theorize that the irrationality seen in the BA system is an emergent property of that system. This conjecture stems from the fact that the same agent behaviour, when simulated with different underlying social networks, leads to radically different global system behaviour. Whereas *figure 1* is characterized by instability and unpredictability, the other models are characterized by much more stable behaviour after the first collapse. 

As the systems display different properties with different social networks, despite the agents behaving and interacting in the same way, this suggests that these differing properties are emergent to each system. Thus the level of rationality and stability seen in each system is likey an emergent property of that system, rather than being an intrinsic property characteristic of the local agent behaviour. 

Finally, all three systems contained at least one bubble, consisting of a rapid expansion followed by a rapid collapse (just like real world market bubbles). This is likely because all 3 systems model "trend-following" behaviour. Trend following refers to entering/leaving a market based on the stock price trends, rather than the intrinsic value of the asset, or an actual qualitative analysis of the company whose shares are being traded. 

Regardless of the social model chosen, the agents in this model all act purely based on observing the market, and have no consideration of intrinsic value. The results suggest that such behaviour almost always results in bubbles. 

This is a sensible suggestion, as a consideration of intrinsic value would be an inhibiting factor in preventing market prices from exceeding them. Removing that can easily drive prices beyond what they should intrinsically reach, and the unsustainability of this will inevitably lead to a collapse. However, following the first collapse, further bubbles are only seen in the BA model. In the other two models, the apprehension investors have to rejoin the market (after previously having left) results in a sort of stability, which remains for the duration of the simulation (due to the lack of external influences that would exist in the real world).